In [1]:
import numpy as np
import pandas as pd
import os
import geopandas as gpd
from scipy import stats
import scipy.optimize

import time
import powerlaw
import pickle5 as pickle
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from pymoo.core.problem import Problem

In [2]:
#Unpickling the data
def load_obj(name):
    with open('resources/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [5]:
class MyProblem(Problem):

    def __init__(self):
        super().__init__(n_var=1, n_obj=1, n_ieq_constr=2, xl=0.0, xu=1.0)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = np.sum((x - 0.5) ** 2, axis=1)
        out["G"] = 0.1 - out["F"]

In [3]:
# Bus adjacency
def bus_adjacency(stoproute,lsoa_list,route_freqs):
    # Create matrix that combines location data and route frequencies
    combine = pd.merge(stoproute, route_freqs, on='line')
    combine = combine.drop_duplicates(['line', 'naptan_sto'])
    combine = combine.rename(columns={'geo_code':'lsoa11cd'})

    # Create adjacency matrix LSOA x route
    bstopfreq = combine[['lsoa11cd', 'naptan_sto', 'line', 'average']]
    adj = pd.pivot(bstopfreq,index=["lsoa11cd", "naptan_sto"], columns="line", values="average").fillna(0)
    adj = adj.astype(float)
    adj = adj.groupby(level="lsoa11cd").mean()
    bus2route = pd.merge(lsoa_list, adj, how='left',on='lsoa11cd').set_index('lsoa11cd')

    #Adjacency matrix LSOA x LSOA
    bus2route = np.array(bus2route)
    bus2routeT = bus2route.transpose()
    lsoa2lsoa = np.dot(bus2route,bus2routeT)**0.5 #check that this actually does whay I think it does
    lsoa2lsoa[np.diag_indices_from(lsoa2lsoa)] = 0

    lsoa2lsoa = pd.DataFrame(lsoa2lsoa)
    lsoa2lsoa = lsoa2lsoa.fillna(0)

    #m values created
    m_bus = np.round(lsoa2lsoa.copy(),0)
    m_bus[m_bus>0]=np.log10(m_bus[m_bus>0])
    m_bus=1-(m_bus/np.max(np.max(m_bus)))
    m_bus[m_bus==0]=np.min(np.min(m_bus[m_bus!=0]))
    
    return m_bus.values

In [4]:
# Attractivity samples - median/neighbourhood, directional 
def attractivity_median_sampler(oa, edu_ratios, income_params, size):
   
    edu = np.random.choice(4, size = size, p=edu_ratios[oa]) #where p values are effectively the ratio of people with a given education level
    income = stats.beta.rvs(income_params[oa, 0], income_params[oa, 1], loc = income_params[oa, 2], scale = income_params[oa, 3], size=size)

    attractivity = np.power(income, -edu)

    return np.median(attractivity)



# Median att matrix 
def median_attractivity(edu_ratios, income_params): #,fit = None):

    """
    Average individual attractivity / lsoa (taken as a sample of 1000 ppl)
    Sample is directinal - matrix not symmetrical
    """

    attractivity = np.zeros((len(income_params)))
    size = 10000

    for i in range(len(income_params)):
        attractivity[i] = attractivity_median_sampler(i, edu_ratios, income_params, size)

    attractivity = attractivity.reshape((len(attractivity),1))

    return attractivity


In [5]:
# Input data 
lsoa_data = load_obj("newdata_lsoa_data")
sheff_shape, income_params, edu_counts, edu_ratios = lsoa_data['sheff_lsoa_shape'], lsoa_data['income_params'], lsoa_data['edu_counts'], lsoa_data['edu_ratios']

# k
comp_ratio = np.load("resources/newdata_companyhouse.npy")

# distances
paths_matrix = load_obj("newdata_ave_paths")
# removes all 0s not on the diag 
paths_matrix[paths_matrix==0] = 1
paths_matrix[np.diag_indices_from(paths_matrix)] = 0

# bus freq paths 
stoproute = pd.read_csv('resources/stoproute_withareacodes.csv')
lsoa_list = pd.read_csv("resources/E47000002_KS101EW.csv")['lsoa11cd']
route_freqs = pd.read_csv('resources/Bus_routes_frequency.csv', usecols= ["line","average"]).astype(str)
m_paths = bus_adjacency(stoproute, lsoa_list, route_freqs)


/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)
/Users/ioana.jelea/opt/anaconda3/envs/lsoa_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: In a future version, DataFrame.min(axis=None) will return a scalar min over the entire DataFrame. To retain the old behavior, use 'frame.min(axis=0)' or just 'frame.min()'
  return reduction(axis=axis, out=out, **passkwargs)


In [6]:
attractivity_avg = median_attractivity(edu_ratios, income_params)

#population amplification
pop = np.asarray(edu_counts).reshape((len(edu_counts), 1))
pop = np.matmul(pop, pop.transpose())

#connectivity matrix
attractivity_product = np.matmul(attractivity_avg, attractivity_avg.transpose())
attractivity_product = np.multiply(attractivity_product, comp_ratio)


/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56306/2752484946.py:13: RuntimeWarning: divide by zero encountered in true_divide
  connectivity = np.divide(attractivity_product, np.power(paths_matrix, m_paths))
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56306/2752484946.py:13: RuntimeWarning: invalid value encountered in true_divide
  connectivity = np.divide(attractivity_product, np.power(paths_matrix, m_paths))


In [136]:
len(paths_matrix[paths_matrix==0])

853

In [139]:
np.sum(np.sum(connectivity, 1),0)

2678803.3980456237

In [147]:
np.sum(np.sum(connectivity))

2678803.398045624

In [10]:
# Example Problem 

import numpy as np
from pymoo.core.problem import ElementwiseProblem

class MyProblem(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var=2,
                         n_obj=2,
                         n_ieq_constr=2,
                         xl=np.array([-2,-2]),
                         xu=np.array([2,2]))

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = 100 * (x[0]**2 + x[1]**2)
        f2 = (x[0]-1)**2 + x[1]**2

        g1 = 2*(x[0]-0.1) * (x[0]-0.9) / 0.18
        g2 = - 20*(x[0]-0.4) * (x[0]-0.6) / 4.8

        out["F"] = [f1, f2]
        out["G"] = [g1, g2]


problem = MyProblem()

In [155]:
# test function syntax
f = np.divide(attractivity_product, np.power(paths_matrix, m_paths)) # needs removing of inf/nan
f[np.where(np.isinf(f))[0], np.where(np.isinf(f))[1]] = 0
f[np.diag_indices_from(f)] = 0
f1 = - np.sum(np.sum(f))

g1 = np.sum(np.sum(m_paths)) - 1.05 * np.sum(np.sum(m_paths))
g2 = 0.95 * np.sum(np.sum(m_paths)) - np.sum(np.sum(m_paths))

f1, g1, g2

/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_46768/2135418558.py:1: RuntimeWarning: divide by zero encountered in true_divide
  f = np.divide(attractivity_product, np.power(paths_matrix, m_paths)) # needs removing of inf/nan
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_46768/2135418558.py:1: RuntimeWarning: invalid value encountered in true_divide
  f = np.divide(attractivity_product, np.power(paths_matrix, m_paths)) # needs removing of inf/nan


(-2678803.398045624, -35120.70885570184, 667293.4682583341)

In [158]:
len(m_paths)**2, 853*853

(727609, 727609)

# Pymoo method

In [159]:
import numpy as np
from pymoo.core.problem import Problem

class MyProblem(Problem):

    def __init__(self):
        super().__init__(n_var=len(m_paths)**2,
                         n_obj=1,
                         n_ieq_constr=2,
                         xl=np.array([0]),
                         xu=np.array([2]))

    def _evaluate(self, m, out, *args, **kwargs):
        f = np.divide(attractivity_product, np.power(paths_matrix, m)) # needs removing of inf/nan
        f[np.where(np.isinf(f))[0], np.where(np.isinf(f))[1]] = 0
        f[np.diag_indices_from(f)] = 0
        f1 = - np.sum(np.sum(f)) 

        g1 = np.sum(np.sum(m)) - 1.05 * np.sum(np.sum(m_paths))
        g2 = 0.95 * np.sum(np.sum(m_paths)) - np.sum(np.sum(m))

        out["F"] = [f1]
        out["G"] = [g1, g2]


problem = MyProblem()


In [160]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling

algorithm = NSGA2(
    pop_size=len(m_paths)**2,
#     n_offsprings=10,
#     sampling=FloatRandomSampling(),
#     crossover=SBX(prob=0.9, eta=15),
#     mutation=PM(eta=20), 
#     eliminate_duplicates=True
)


In [161]:
from pymoo.core.termination import NoTermination

termination = NoTermination() #termination condition needed, kernel died otherwise 

In [ ]:
from pymoo.optimize import minimize

res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               save_history=True,
               verbose=True)

X = res.X
F = res.F

# Test optimiser method - def

In [21]:
from scipy.optimize import minimize

# objective function - sum of connectivity
def rosen(x):
    """The Rosenbrock function"""
    return sum(100.0*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0)

# calculating the gradient of above function 
def rosen_der(x):
    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    der = np.zeros_like(x)
    der[1:-1] = 200*(xm-xm_m1**2) - 400*(xm_p1 - xm**2)*xm - 2*(1-xm)
    der[0] = -400*x[0]*(x[1]-x[0]**2) - 2*(1-x[0])
    der[-1] = 200*(x[-1]-x[-2]**2)
    return der

from scipy.optimize import Bounds
bounds = Bounds([0, -0.5], [1.0, 2.0])

In [22]:
ineq_cons = {'type': 'ineq',
              'fun' : lambda x: np.array([1 - x[0] - 2*x[1],
                                          1 - x[0]**2 - x[1],
                                          1 - x[0]**2 + x[1]]),
              'jac' : lambda x: np.array([[-1.0, -2.0],
                                          [-2*x[0], -1.0],
                                          [-2*x[0], 1.0]])}   #derivation from fun
eq_cons = {'type': 'eq',
            'fun' : lambda x: np.array([2*x[0] + x[1] - 1]),
            'jac' : lambda x: np.array([2.0, 1.0])}

In [23]:
x0 = np.array([0.5, 0])
>>> res = minimize(rosen, x0, method='SLSQP', jac=rosen_der,
...                constraints=[eq_cons, ineq_cons], options={'ftol': 1e-9, 'disp': True})
...                #bounds=bounds)
# may vary

>>> print(res.x)


Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.34271757499419825
            Iterations: 4
            Function evaluations: 5
            Gradient evaluations: 4
[0.41494475 0.1701105 ]


In [85]:
np.log(attractivity_product)

/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_46768/1522794454.py:1: RuntimeWarning: divide by zero encountered in log
  np.log(attractivity_product)


array([[2.61021563, 3.13270364, 3.68139844, ..., 3.12391036, 2.95254481,
        4.3106923 ],
       [3.13270364, 3.28685804, 3.83555284, ..., 3.29686221, 3.1459901 ,
        4.4648467 ],
       [3.68139844, 3.83555284, 4.38424763, ..., 3.84555701, 3.6946849 ,
        5.01354149],
       ...,
       [3.12391036, 3.29686221, 3.84555701, ..., 3.28806893, 3.13719682,
        4.47485087],
       [2.95254481, 3.1459901 , 3.6946849 , ..., 3.13719682, 2.96583127,
        4.32397876],
       [4.3106923 , 4.4648467 , 5.01354149, ..., 4.47485087, 4.32397876,
        5.49763436]])

In [86]:
attractivity_product

array([[ 13.60198359,  22.93590632,  39.7018759 , ...,  22.73510856,
         19.15463669,  74.49204166],
       [ 22.93590632,  26.75865693,  46.31902756, ...,  27.02769859,
         23.24267674,  86.90770531],
       [ 39.7018759 ,  46.31902756,  80.17787738, ...,  46.78473659,
         40.23289312, 150.43656367],
       ...,
       [ 22.73510856,  27.02769859,  46.78473659, ...,  26.79107828,
         23.03919329,  87.78150825],
       [ 19.15463669,  23.24267674,  40.23289312, ...,  23.03919329,
         19.41083219,  75.48838141],
       [ 74.49204166,  86.90770531, 150.43656367, ...,  87.78150825,
         75.48838141, 244.11376392]])

# Implementation for bus network

**(done)** remove all 0s not on diag from paths_matrix and replace with 1

**(done)** pymoo - use 853 x 853 as number of variables 

**(done)** run optimizer without the jacobian mat 

**(done)** reshape m inside the function so that it is a 853 x 853 istead of a string - not rly working, maybe too big?

In [7]:
# A_NEW = A[start_index_row : stop_index_row, 
#           start_index_column : stop_index_column]

attractivity_p = attractivity_product[0:9,0:9]
paths_mat = paths_matrix[0:9,0:9]
m_pat = m_paths[0:9,0:9]

In [9]:
from scipy.optimize import minimize

# test run for smaller matrix
def m_opt(m):
    
    m = np.reshape(m,(len(m_pat),len(m_pat)))
    f = np.divide(attractivity_p, np.power(paths_mat, m)) # needs removing of inf/nan
    f[np.where(np.isinf(f))[0], np.where(np.isinf(f))[1]] = 0
    f[np.diag_indices_from(f)] = 0
    f1 = - np.sum(np.sum(f))
    
    return f1

# # calculating the gradient of above function 
# def opt_der(m):
#     m = np.reshape(m,(len(m_pat),len(m_pat)))
#     der = np.zeros_like(m) # don't think it's needed 
#     der = np.multiply(attractivity_p, np.power(paths_mat, -m), -m)
#     der = sum(sum(np.multiply(np.log(attractivity_p),der)))
#     return der


ineq_cons = {'type': 'ineq',
              'fun' : lambda m: np.array([-1.05 * np.sum(np.sum(m_pat)) + np.sum(np.sum(m)),   #A1*sum M - sum m
                                          -np.sum(np.sum(m)) + 0.95 * np.sum(np.sum(m_paths))])}  #sum m - A2*sum M
              #'jac' : lambda m: np.array([[-1.0],
                                          #[1.0]])}
eq_cons = {'type': 'eq',
            'fun' : lambda m: np.array(np.median(m)),
            #'jac' : lambda m: np.array(0)}  #??


x0 = np.random.rand(len(m_pat),len(m_pat))
res = minimize(m_opt, x0, method='SLSQP', #jac=opt_der,
            constraints=[eq_cons, ineq_cons], options={'ftol': 1e-9, 'disp': True})
            #bounds=bounds)
# may vary

print(res.x)


SyntaxError: invalid syntax (2812182226.py, line 33)

In [44]:
m_pat

array([[1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 0.16017428, 0.1168138 , 0.30491597,
        0.1696565 , 1.        , 1.        , 1.        ],
       [1.        , 0.16017428, 1.        , 0.16017428, 0.44164589,
        0.21301698, 1.        , 1.        , 1.        ],
       [1.        , 0.1168138 , 0.16017428, 1.        , 0.30491597,
        0.1696565 , 1.        , 1.        , 1.        ],
       [1.        , 0.30491597, 0.44164589, 0.30491597, 1.        ,
        0.35775867, 1.        , 1.        , 1.        ],
       [1.        , 0.1696565 , 0.21301698, 0.1696565 , 0.35775867,
        1.        , 1.16648938, 1.20129597, 1.16648938],
       [1.        , 1.        , 1.        , 1.        , 1.        ,
        1.16648938, 1.        , 0.2170458 , 0.23296786],
       [1.        , 1.        , 1.        , 1.        , 1.        ,
        1.20129597, 0.2170458 , 1.        , 0.23298566],


In [10]:
from scipy.optimize import minimize

# objective function - sum of connectivity
def m_opt(m):
    
    m = np.reshape(m,(len(m_paths),len(m_paths)))
    f = np.divide(attractivity_product, np.power(paths_matrix, m)) # needs removing of inf/nan
    f[np.where(np.isinf(f))[0], np.where(np.isinf(f))[1]] = 0
    f[np.diag_indices_from(f)] = 0
    f1 = - np.sum(np.sum(f))
    
    return f1

# don't use below for now

# # calculating the gradient of above function 
# def opt_der(m):
#     der = np.zeros_like(x) # don't think it's needed 
#     der = np.multiply(attractivity_product, np.power(paths_matrix, -m), -m)
#     der = sum(sum(np.multiply(np.log(attractivity_product),der)))
#     return der

# # can try without the bounds - not change from rosenbrock
# from scipy.optimize import Bounds
# bounds = Bounds([0, -0.5], [1.0, 2.0])

In [11]:
ineq_cons = {'type': 'ineq',
              'fun' : lambda m: np.array([1.05 * np.sum(np.sum(m_paths)) - np.sum(np.sum(m)),   #A1*sum M - sum m
                                          np.sum(np.sum(m)) - 0.95 * np.sum(np.sum(m_paths))])} #sum m - A2*sum M
              #'jac' : lambda x: np.array([[-1.0],
                                          #[1.0]])}
eq_cons = {'type': 'eq',
            'fun' : lambda m: np.array(np.median(m))}
            #'jac' : lambda x: np.array(0)}  #??

# try with 2 ineqs as well?

In [ ]:
x0 = np.ones_like(m_paths)
res = minimize(m_opt, x0, method='SLSQP', #jac=opt_der,
            constraints=[eq_cons, ineq_cons], options={'ftol': 1e-9, 'disp': True})
            #bounds=bounds)
# may vary

print(res.x)


/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56306/3365054683.py:7: RuntimeWarning: divide by zero encountered in true_divide
  f = np.divide(attractivity_product, np.power(paths_matrix, m)) # needs removing of inf/nan
/var/folders/9v/cwjxgbc17v9gt368p99ndkw80000gn/T/ipykernel_56306/3365054683.py:7: RuntimeWarning: invalid value encountered in true_divide
  f = np.divide(attractivity_product, np.power(paths_matrix, m)) # needs removing of inf/nan


In [26]:
x0

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [31]:
m_paths

array([[1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 1.        , 0.16017428, ..., 1.        , 1.        ,
        1.        ],
       [1.        , 0.16017428, 1.        , ..., 1.        , 1.        ,
        1.        ],
       ...,
       [1.        , 1.        , 1.        , ..., 1.        , 0.01653876,
        1.        ],
       [1.        , 1.        , 1.        , ..., 0.01653876, 1.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ]])